# Binary Classification Metrics implementation using numpy

In this notebook I will implement some binary classification metrics like **accuracy**,**precision**,**recall** and **roc auc** using *numpy* only and then compare my implementation with *sklearn's* implementation

## Confusion Matrix

|Label\Prediction             | Positive Prediction | Negative Prediction         |
| ---                         |    ---              |    ---                      |
| **Positive Label**              | *TP*                | *FN*                        |
| **Negative Label**              | *FP*                | *TN*                        |

<img src="https://latex.codecogs.com/png.latex?Precision&space;=\frac{tp}{tp&plus;fp}" title="Precision =\frac{tp}{tp+fp}" />
<img src="https://latex.codecogs.com/png.latex?Recall&space;=\frac{tp}{tp&plus;fn}" title="Recall =\frac{tp}{tp+fn}"/>


## Receiver Operator Curve
![Receiver Operator Curve(ROC)](assets/roc.png)

In [1]:
import numpy as np
#Some assertions to check equal array sizes and to check that it is binary classification problem
def assert_sizes(y_true,y_pred):
    
    assert(y_true.shape == y_pred.shape)
    assert(np.unique(y_true).shape[0] == 2)
    assert(np.max(y_true) == 1 and np.min(y_true)==0)
    
def get_y_thresholded(y_pred,threshold):
    y_thresholded=np.copy(y_pred)
    y_thresholded[y_thresholded >= threshold]=1
    y_thresholded[y_thresholded < threshold]=0
    return y_thresholded

def my_accuracy(y_true,y_pred,threshold=0.5):
    
    assert_sizes(y_true,y_pred)
    y_thresholded = get_y_thresholded(y_pred,threshold)
    return np.sum(y_true == y_thresholded)/len(y_true)

def my_precision(y_true,y_pred,threshold=0.5):
    assert_sizes(y_true,y_pred)
    y_thresholded = get_y_thresholded(y_pred,threshold)
    condition1,condition2,condition3,condition4 = y_true==y_thresholded,y_true !=y_thresholded,y_true == 1,y_true == 0
    tp = np.sum(np.logical_and(condition1,condition3))
    fp = np.sum(np.logical_and(condition2,condition4))
    
    return tp/(tp+fp)

def my_recall(y_true,y_pred,threshold=0.5):
    assert_sizes(y_true,y_pred)
    y_thresholded = get_y_thresholded(y_pred,threshold)
    condition1,condition2,condition3,condition4 = y_true==y_thresholded,y_true !=y_thresholded,y_true == 1,y_true == 0
    tp = np.sum(np.logical_and(condition1,condition3))
    fn = np.sum(np.logical_and(condition2,condition3))
    
    return tp/(tp+fn)
######################################################################################################################
# Example Showing how ROC is drawn
######################################################################################################################

# <-|-x---x--o-x-o x----x---o--x-->

# tpr = true_positives(number of positives on the right of the separator)/all_positives
# fpr = false_positives(number of negatives on the right of the separator)/all_negatives

# tpr = 3/3 =1
# fpr = 6/6 =1

#(1,1)

# <---x-|-x--o-x-o x----x---o--x-->

# tpr = 3/3 =1
# fpr = 5/6
#(5/6,1)

# <---x---x|-o-x-o x----x---o--x-->

#tpr = 3/3 =1
#fpr = 4/6 =2/3

#And so on then draw these points from bottom to top

# then area under the curve is calculated by using trapezium rule
def my_auc_roc(y_true,y_pred,threshold=0.5):
    assert_sizes(y_true,y_pred)
    y_pred_indices_sorted=np.argsort(y_pred)[::-1]
    y_true = y_true[y_pred_indices_sorted]
    all_positives = np.sum(y_true == 1)
    all_negatives = np.sum(y_true == 0)
    tps=np.cumsum(y_true)
    fps=(np.arange(len(y_true))+1) - tps
    tpr = tps / all_positives
    fpr = fps / all_negatives
    
    return np.trapz(tpr,fpr)
    
    

## Sanity Checks
> Now it's time to test our own implementation of binary metrics with sklearn

In [2]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score
np.random.seed(42)
y_true = np.array([1,0,1,1,0,0,1,0,1])
y_pred = np.random.uniform(0,1,len(y_true))
y_thresholded = get_y_thresholded (y_pred,0.5)
#check correctness of implementation of accuracy
assert (accuracy_score(y_true,y_thresholded) == my_accuracy(y_true,y_pred))

#check correctness of implementation of precision
assert (precision_score(y_true,y_thresholded) == my_precision(y_true,y_pred))

#check correctness of implementation of recall
assert (recall_score(y_true,y_thresholded) == my_recall(y_true,y_pred))

#check correctness of implementation of auc

assert (roc_auc_score(y_true,y_pred) == my_auc_roc(y_true,y_pred))

print ('All Correct')

All Correct
